## Import Libraries

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from faker import Faker
import random
np.random.seed()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from model import *
from utils import *

## Create Dataset

In [9]:
x = []
palindromes = []
for i in range(1024):
    binary_string = format(i, '010b')
    x.append(binary_string)
    if binary_string == binary_string[::-1]:
        palindromes.append(binary_string)
x = np.array(x)
palindromes = np.array(palindromes)


In [10]:
print(len(palindromes))
palindromes

32


array(['0000000000', '0000110000', '0001001000', '0001111000',
       '0010000100', '0010110100', '0011001100', '0011111100',
       '0100000010', '0100110010', '0101001010', '0101111010',
       '0110000110', '0110110110', '0111001110', '0111111110',
       '1000000001', '1000110001', '1001001001', '1001111001',
       '1010000101', '1010110101', '1011001101', '1011111101',
       '1100000011', '1100110011', '1101001011', '1101111011',
       '1110000111', '1110110111', '1111001111', '1111111111'],
      dtype='<U10')

In [11]:
np.random.shuffle(palindromes)
print(len(palindromes))
palindromes

32


array(['0110110110', '1101001011', '0010110100', '0001001000',
       '0110000110', '1011111101', '0100000010', '1010110101',
       '0000000000', '0011111100', '1001001001', '0011001100',
       '0111001110', '1011001101', '1001111001', '1000110001',
       '1000000001', '0101001010', '1110110111', '0001111000',
       '0111111110', '1110000111', '0010000100', '1101111011',
       '0101111010', '1100110011', '1111001111', '0100110010',
       '0000110000', '1100000011', '1111111111', '1010000101'],
      dtype='<U10')

In [126]:
for i in range(5):
    x = np.concatenate((x, palindromes))
print(len(x))

1184


In [127]:
y = []
for binary_string in x:
    y.append(binary_string == binary_string[::-1])
y = np.array(y)
permutation_index = np.random.permutation(len(x))
x = x[permutation_index]
y = y[permutation_index]

In [128]:
def sigmoid(x):
    return 1 / (1 + np.exp(-1 * x))

def relu(x):
    return np.maximum(0, x)

In [129]:
# a = np.array([1, 2, -3])
# b = np.array([2, 3, 4])
# a*b

array([  2,   6, -12])

In [13]:
LEARNING_RATE = 1e-3
NUM_EPOCHS = 10000

input_size = 10
hidden_layer_size = 20
output_size = 1

weights_ih = np.random.rand(input_size, hidden_layer_size)
weights_ho = np.random.rand(hidden_layer_size, output_size)

In [ ]:
x = np.array([])

In [131]:
# LEARNING_RATE = 1e-7
for epoch in range(NUM_EPOCHS):
    tot_loss = 0
    for i, s in enumerate(x):
        inp = np.reshape(np.array([int(char) for char in s]), (1, input_size))
        hlayer_logits = np.dot(inp, weights_ih)
        hlayer_output = sigmoid(hlayer_logits)

        final_logits = np.dot(hlayer_output, weights_ho)
        final_output = sigmoid(final_logits)

        tot_loss += abs(final_output - y[i])

        output_delta = (y[i] - final_output) * final_output * (1 - final_output)

        weights_ho += LEARNING_RATE * hlayer_output.T.dot(output_delta)
        weights_ih += LEARNING_RATE * inp.T.dot(output_delta.dot(weights_ho.T) * hlayer_output * (1 - hlayer_output))

    if epoch % 100 == 0:
        print(f'Epoch {epoch} : loss {tot_loss}')

Epoch 0 : loss [[991.61380209]]
Epoch 100 : loss [[989.2316454]]
Epoch 200 : loss [[319.53270712]]
Epoch 300 : loss [[319.57421463]]
Epoch 400 : loss [[319.61364199]]
Epoch 500 : loss [[319.6512102]]
Epoch 600 : loss [[319.6870946]]
Epoch 700 : loss [[319.72143354]]
Epoch 800 : loss [[319.75433739]]
Epoch 900 : loss [[319.78589815]]
Epoch 1000 : loss [[319.81620019]]
Epoch 1100 : loss [[319.84533201]]
Epoch 1200 : loss [[319.87339917]]
Epoch 1300 : loss [[319.90053773]]
Epoch 1400 : loss [[319.92692755]]
Epoch 1500 : loss [[319.9528046]]
Epoch 1600 : loss [[319.97847091]]
Epoch 1700 : loss [[320.00430116]]
Epoch 1800 : loss [[320.03074492]]
Epoch 1900 : loss [[320.05832377]]
Epoch 2000 : loss [[320.08762303]]
Epoch 2100 : loss [[320.11927799]]
Epoch 2200 : loss [[320.15395461]]
Epoch 2300 : loss [[320.19232432]]
Epoch 2400 : loss [[320.23503171]]
Epoch 2500 : loss [[320.28265285]]
Epoch 2600 : loss [[320.33564028]]
Epoch 2700 : loss [[320.39424877]]
Epoch 2800 : loss [[320.45843371]]
E

In [139]:
LEARNING_RATE = 1e-1
for epoch in range(NUM_EPOCHS):
    tot_loss = 0
    for i, s in enumerate(x):
        inp = np.reshape(np.array([int(char) for char in s]), (1, input_size))
        hlayer_logits = np.dot(inp, weights_ih)
        hlayer_output = sigmoid(hlayer_logits)

        final_logits = np.dot(hlayer_output, weights_ho)
        final_output = sigmoid(final_logits)

        tot_loss += abs(final_output - y[i])

        output_delta = (y[i] - final_output) * final_output * (1 - final_output)

        weights_ho += LEARNING_RATE * hlayer_output.T.dot(output_delta)
        weights_ih += LEARNING_RATE * inp.T.dot(output_delta.dot(weights_ho.T) * hlayer_output * (1 - hlayer_output))

    if epoch % 100 == 0:
        print(f'Epoch {epoch} : loss {tot_loss}')

Epoch 0 : loss [[0.70983834]]
Epoch 100 : loss [[0.7091767]]
Epoch 200 : loss [[0.70852412]]
Epoch 300 : loss [[0.70787714]]
Epoch 400 : loss [[0.70723423]]
Epoch 500 : loss [[0.70659466]]
Epoch 600 : loss [[0.70595803]]


KeyboardInterrupt: 

In [143]:
x_test1 = '1100110011'
x_test2 = '1100110000'
def test(x):
    inp = np.reshape(np.array([int(char) for char in x]), (1, input_size))
    return sigmoid(np.dot(sigmoid(np.dot(inp, weights_ih)), weights_ho))

print(test(x_test1))
print(test(x_test2))

[[0.99802693]]
[[9.87927878e-05]]


In [144]:
# np.save('weights_ih.npy', weights_ih)
# np.save('weights_ho.npy', weights_ho)

In [1]:
from utils import activation, dataset, losses
from model import *
import numpy as np

Dataset = dataset.Palindrome_Dataset(n_bits=10)
x,y = Dataset.get_data(shuffle=False)

In [2]:
model = Palindrome_Model(input_size=1dataset0, output_size=1,hidden_layer_sizes=[4],activation='sigmoid')

In [3]:
model.layers

[{'name': 'layer-1',
  'weights': array([[ 0.08701854, -0.19543489,  1.12983589,  0.04685803],
         [-0.11306758, -0.19295109, -0.36208643, -0.65408091],
         [ 0.89565762,  0.90016614, -0.20350269,  0.35228063],
         [ 1.16677101,  0.06401665,  0.32028525,  0.37577088],
         [ 0.73298734, -0.27092703, -0.048532  ,  0.20221205],
         [-0.76231812, -1.06370107,  0.25430709, -0.00645013],
         [-0.0719853 ,  0.02542935, -0.90182575, -0.91726245],
         [-0.78590996,  0.28289155, -0.28547343,  0.66010028],
         [ 1.55533262, -0.39884791, -0.06661003,  0.20458392],
         [-0.08053847,  0.58702988, -0.37884862,  0.81494488]]),
  'biases': array([-0.35123382, -0.87013436,  0.8724007 , -1.27402972]),
  'activation': 'relu'},
 {'name': 'layer-2',
  'weights': array([[ 0.60919384],
         [ 0.41314292],
         [ 0.26754641],
         [-0.51083802]]),
  'biases': array([-0.73706457]),
  'activation': 'sigmoid'}]

In [4]:
input = x[100]
input

array([0, 0, 0, 1, 1, 0, 0, 1, 1, 1])

In [5]:
model.forward(input)

KeyError: 'weigths'